In [1]:
# Installs
%pip install -q langchain langchain-community langchain-openai fastembed qdrant-client oxrdflib --quiet

Note: you may need to restart the kernel to use updated packages.


In [88]:
# Imports
from operator import itemgetter
import os

from langchain.globals import set_debug
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import format_document
from langchain_openai import OpenAI
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import get_buffer_string
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.document_loaders.base import BaseLoader
from langchain_core.documents import Document

from rdflib import Graph
from typing import Any, List, Optional
import numpy as np
import pandas as pd

In [89]:
# Read OpenAI key
f = open("../data/openai-key.txt", "r")
lst = f.readlines()
f.close() 

openai_api_key = lst[-1]
os.environ["OPENAI_API_KEY"] = openai_api_key

In [90]:
def _get_class_query() -> str:
    """
    Query to extract class labels
    """
    
    return """
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl:  <http://www.w3.org/2002/07/owl#>
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        PREFIX dc: <http://purl.org/dc/elements/1.1/>
        PREFIX dcterms: <http://purl.org/dc/terms/>

        SELECT ?uri ?pred ?label ?type
        WHERE {
            ?uri a ?type ;
                ?pred ?label .
            FILTER (
                ?type = owl:Class
            )
            FILTER (
                ?pred = rdfs:label ||
                ?pred = skos:prefLabel ||
                ?pred = skos:altLabel ||
                ?pred = skos:definition ||
                ?pred = rdfs:comment ||
                ?pred = dcterms:description ||
                ?pred = dc:title
            )
        }
    """


def _get_property_query() -> str:
    """
    Query to extract property labels
    """
    
    return """
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl:  <http://www.w3.org/2002/07/owl#>
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        PREFIX dc: <http://purl.org/dc/elements/1.1/>
        PREFIX dcterms: <http://purl.org/dc/terms/>

        SELECT ?uri ?pred ?label ?type
        WHERE {
            ?uri a ?type ;
                ?pred ?label .
            FILTER (
                ?type = owl:DatatypeProperty ||
                ?type = owl:ObjectProperty
            )
            FILTER (
                ?pred = rdfs:label ||
                ?pred = skos:prefLabel ||
                ?pred = skos:altLabel ||
                ?pred = skos:definition ||
                ?pred = rdfs:comment ||
                ?pred = dcterms:description ||
                ?pred = dc:title
            )
        }
    """


class OntologyLoader(BaseLoader):
    """
    Load an OWL ontology and extract classes and properties as documents.
    """

    def __init__(self, ontology_url: str, format: Optional[str] = None):
        """
        Initialize the OntologyLoader.

        Args:
            ontology_url (str): URL of the OWL ontology to be loaded.
            format (str): Format of the OWL ontology to be loaded.
        """

        self.ontology_url = ontology_url
        self.format = format
        self.graph = Graph(store="Oxigraph")

    def load(self) -> List[Document]:
        """
        Load and return documents (classes and properties) from the OWL ontology.
        """

        if self.format:
            self.graph.parse(self.ontology_url, format=self.format)
        else:
            self.graph.parse(self.ontology_url)

        # Extract classes and properties as documents
        docs: List[Document] = []
        for cls in self.graph.query(_get_class_query()):
            docs.append(self._create_document(cls))
        for prop in self.graph.query(_get_property_query()):
            docs.append(self._create_document(prop))
        return docs

    def _create_document(self, result_row: Any) -> Document:
        """
        Create a Document object from a query result row.
        """
        
        label = str(result_row.label)
        return Document(
            page_content=label,
            metadata={
                "label": label,
                "uri": str(result_row.uri),
                "type": str(result_row.type),
                "predicate": str(result_row.pred),
                "ontology": self.ontology_url,
            },
        )

In [91]:
def prep_retreiver(embed_name="BAAI/bge-small-en-v1.5",
                    embed_max_length=512,
                    ontology_url="../data/health.ttl",
                    ontology_format="ttl",
                    split_size=1000,
                    split_overlap=200,
                    k = 45):

    flag_embeddings = FastEmbedEmbeddings(model_name=embed_name, max_length=embed_max_length)
    loader = OntologyLoader(ontology_url=ontology_url, format=ontology_format)
    docs = loader.load()

    # Split the documents into chunks if necessary
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=split_size, chunk_overlap=split_overlap)
    splits = text_splitter.split_documents(docs)

    vectorstore = Qdrant.from_documents(
        splits,
        flag_embeddings,
        collection_name="ontologies",
        location=":memory:",
    )

    # K is the number of source documents retrieved
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})

    return retriever

def prep_llm(temp=0):
    llm = OpenAI(temperature=temp)
    return llm

def prep_memory():
    # Create the memory object that is used to add messages
    memory = ConversationBufferMemory(return_messages=True, output_key="answer", input_key="question")

    # Add a "memory" key to the input object
    loaded_memory = RunnablePassthrough.assign(chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),)

    return memory, loaded_memory

def prep_prompts(reform_template, answer_template):
    REFORM_QUESTION_PROMPT = PromptTemplate.from_template(reform_template)
    ANSWER_PROMPT = ChatPromptTemplate.from_template(answer_template)
    DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="Concept label: {page_content} | URI: {uri} | Type: {type} | Predicate: {predicate} | Ontology: {ontology}")

    return REFORM_QUESTION_PROMPT, ANSWER_PROMPT, DEFAULT_DOCUMENT_PROMPT

def _combine_documents(docs, document_prompt, document_separator="\n\n"):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

def prep_chain(REFORM_QUESTION_PROMPT, llm, retriever, DEFAULT_DOCUMENT_PROMPT, ANSWER_PROMPT, loaded_memory):

    # Reformulate the question using chat history
    reformulated_question = {
        "reformulated_question": {
            "question": lambda x: x["question"],
            "chat_history": lambda x: get_buffer_string(x["chat_history"]),
        }
        | REFORM_QUESTION_PROMPT
        | llm
        | StrOutputParser(),
    }

    # Retrieve the documents using the reformulated question
    retrieved_documents = {
        "docs": itemgetter("reformulated_question") | retriever,
        "question": lambda x: x["reformulated_question"] #or print("💭 Reformulated question:", x["reformulated_question"]),
    }

    # Construct the inputs for the final prompt using retrieved documents
    final_inputs = {
        "context": lambda x: _combine_documents(x["docs"], DEFAULT_DOCUMENT_PROMPT),
        "question": itemgetter("question"),
    }

    # Generate the answer using the retrieved documents and answer prompt
    answer = {
        "answer": final_inputs | ANSWER_PROMPT | llm,
        "docs": itemgetter("docs"),
    }

    # Put the chain together
    final_chain = loaded_memory | reformulated_question | retrieved_documents | answer

    return final_chain

def stream_chain(final_chain, memory: ConversationBufferMemory, memoryless, inputs: dict[str, str]) -> dict[str, Any]:
    """
    Ask question, stream the answer output, and return the answer with source documents.
    """
    
    output = {"answer": ""}
    
    for chunk in final_chain.stream(inputs):
                
        if "docs" in chunk:
            output["docs"] = [doc.dict() for doc in chunk["docs"]]
            #print("📚 Documents retrieved:")
            
            for doc in output["docs"]:
                #print(f"· {doc['page_content']} ({doc['metadata']['uri']})")
                continue

        if "answer" in chunk:
            output["answer"] += chunk["answer"]
            #print(chunk["answer"], end="", flush=True)

    if memoryless == 0:
        memory.save_context(inputs, {"answer": output["answer"]})

    return output

In [92]:
def multi_hop(subject, q_lst, retriever, llm, reform_template, answer_template, memoryless=0):
    """
    According to a given subject and a question list, performs multi-hop reasoning by asking questions in order.

    Returns: Two-column array of questions and their answers.
    """
    q_lst[0] = q_lst[0].format(concept = subject) # Inject the subject into questions

    if memoryless not in [0, 1]: # Check if memorylessness is inputted correctly
        print("Input memoryless parameter as binary.")
        return
    
    memory, loaded_memory = prep_memory() # Create empty memory
    REFORM_QUESTION_PROMPT, ANSWER_PROMPT, DEFAULT_DOCUMENT_PROMPT = prep_prompts(reform_template, answer_template)
    final_chain = prep_chain(REFORM_QUESTION_PROMPT, llm, retriever, DEFAULT_DOCUMENT_PROMPT, ANSWER_PROMPT, loaded_memory)
    final = []

    for q in q_lst:
        output = stream_chain(final_chain=final_chain, memory=memory, memoryless=memoryless, inputs={"question": q}) # Get answer
        final.append([f"Question: {q}", output["answer"]])
        
    return np.array(final)

In [93]:
# Prompt to reformulate the question using the chat history
reform_template = """Given the following chat history and a follow up question,
rephrase the follow up question to be a standalone straightforward question, in its original language.
Do not answer the question! Just rephrase reusing informations from the chat history.
Make it short and straight to the point.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:
"""

# Prompt to ask to answer the reformulated question
answer_template = """Briefly answer the question based only on the following context,
do not use any information outside this context:
{context}

Question: {question}
"""

# List of prompts for multi-hop reasoning
q_lst = ["Can you define me what a {concept} is?", "What are similar to this concept?", "What are their objects' URIs with their labels?"]

# Subject to be enquired about
subject = "urethritis"

In [94]:
retriever = prep_retreiver()
llm = prep_llm()

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [95]:
final = multi_hop(subject=subject, q_lst=q_lst, retriever=retriever, llm=llm, reform_template=reform_template, answer_template=answer_template)

In [96]:
for i in final:
    print(i)

['Question: Can you define me what a urethritis is?'
 '\nUrethritis is a type of urethral disease that involves inflammation or infection of the urethra, which is the tube that carries urine from the bladder out of the body. It can be caused by various factors, including bacteria, viruses, and sexually transmitted infections. Symptoms may include pain or burning during urination, frequent urination, and discharge from the urethra.']
['Question: What are similar to this concept?'
 '\nPossible answer: Urethral disease, urethral syndrome, urethral obstruction, Chlamydia trachomatis urethritis, urethral stricture, infective urethral stricture, urethral gland abscess, urogenital abnormality, urinary tract obstruction, urethral calculus, gonococcal urethritis, urethral benign neoplasm, autoimmune disease of urogenital tract, urinary tract infection, urethral false passage, urinary schistosomiasis, pyelonephritis, Trichomonas urethritis, chlamydia, Chlamydia pneumonia, urethral diverticulum, 